<a href="https://colab.research.google.com/github/chobitsX/30-seconds-of-code-zh-CN/blob/master/nb/Qwen2.5_(3B)-GRPO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://docs.unsloth.ai/get-started/installing-+-updating).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save)


### News

**Read our [blog post](https://unsloth.ai/blog/r1-reasoning) for guidance on how to train reasoning models.**

Visit our docs for all our [model uploads](https://docs.unsloth.ai/get-started/all-our-models) and [notebooks](https://docs.unsloth.ai/get-started/unsloth-notebooks).


### Installation

In [9]:
%%capture
# Skip restarting message in Colab
import sys; modules = list(sys.modules.keys())
for x in modules: sys.modules.pop(x) if "PIL" in x or "google" in x else None

!pip install unsloth vllm
!pip install --upgrade pillow
# If you are running this notebook on local, you need to install `diffusers` too
# !pip install diffusers
# Temporarily install a specific TRL nightly version
!pip install git+https://github.com/huggingface/trl.git@e95f9fb74a3c3647b86f251b7e230ec51c64b72b

### Unsloth

Use `PatchFastRL` before all functions to patch GRPO and other RL algorithms!

In [10]:
from unsloth import FastLanguageModel, PatchFastRL
PatchFastRL("GRPO", FastLanguageModel)

Load up `Qwen 2.5 3B Instruct`, and set parameters

In [11]:
from unsloth import is_bfloat16_supported
import torch
max_seq_length = 1024 # Can increase for longer reasoning traces
lora_rank = 64 # Larger rank = smarter, but slower

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "Qwen/Qwen2.5-3B-Instruct",
    max_seq_length = max_seq_length,
    load_in_4bit = True, # False for LoRA 16bit
    fast_inference = True, # Enable vLLM fast inference
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.5, # Reduce if out of memory
)

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ], # Remove QKVO if out of memory
    lora_alpha = lora_rank,
    use_gradient_checkpointing = "unsloth", # Enable long context finetuning
    random_state = 3407,
)

==((====))==  Unsloth 2025.2.5: Fast Qwen2 patching. Transformers: 4.48.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/qwen2.5-3b-instruct-unsloth-bnb-4bit with actual GPU utilization = 33.79%
Unsloth: Your GPU has CUDA compute capability 7.5 with VRAM = 14.74 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 1024. Num Sequences = 160.
Unsloth: vLLM's KV Cache can use up to 2.56 GB. Also swap space = 1 GB.
WARNING 02-12 05:38:07 config.py:2386] Casting torch.bfloat16 to torch.float16.
INFO 02-12 05:38:07 config.py:542] This model supports multiple tasks: {'classify', 'reward', 'score', 'generate', 'embed'}. De

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 02-12 05:38:12 model_runner.py:1115] Loading model weights took 2.2082 GB
INFO 02-12 05:38:14 worker.py:267] Memory profiling takes 1.16 seconds
INFO 02-12 05:38:14 worker.py:267] the current vLLM instance can use total_gpu_memory (14.74GiB) x gpu_memory_utilization (0.34) = 4.98GiB
INFO 02-12 05:38:14 worker.py:267] model weights take 2.21GiB; non_torch_memory takes 0.00GiB; PyTorch activation peak memory takes 0.87GiB; the rest of the memory reserved for KV Cache is 1.91GiB.
INFO 02-12 05:38:14 executor_base.py:110] # CUDA blocks: 3470, # CPU blocks: 1820
INFO 02-12 05:38:14 executor_base.py:115] Maximum concurrency for 1024 tokens per request: 54.22x
INFO 02-12 05:38:15 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_utili

Capturing CUDA graph shapes: 100%|██████████| 23/23 [00:37<00:00,  1.62s/it]

INFO 02-12 05:38:52 model_runner.py:1562] Graph capturing finished in 37 secs, took 0.08 GiB
INFO 02-12 05:38:52 llm_engine.py:431] init engine (profile, create kv cache, warmup model) took 40.52 seconds


### Data Prep
<a name="Data"></a>

We directly leverage [@willccbb](https://gist.github.com/willccbb/4676755236bb08cab5f4e54a0475d6fb) for data prep and all reward functions. You are free to create your own!

In [12]:
import re
from datasets import load_dataset, Dataset

# Load and prep dataset
SYSTEM_PROMPT = """
Respond in the following format:
<reasoning>
...
</reasoning>
<answer>
...
</answer>
"""

XML_COT_FORMAT = """\
<reasoning>
{reasoning}
</reasoning>
<answer>
{answer}
</answer>
"""

def extract_xml_answer(text: str) -> str:
    answer = text.split("<answer>")[-1]
    answer = answer.split("</answer>")[0]
    return answer.strip()

def extract_hash_answer(text: str) -> str | None:
    if "####" not in text:
        return None
    return text.split("####")[1].strip()

# uncomment middle messages for 1-shot prompting
def get_gsm8k_questions(split = "train") -> Dataset:
    data = load_dataset('swulling/gsm8k_chinese', 'default')[split] # type: ignore
    data = data.map(lambda x: { # type: ignore
        'prompt': [
            {'role': 'system', 'content': SYSTEM_PROMPT},
            {'role': 'user', 'content': x['question_zh-cn']}
        ],
        'answer': extract_hash_answer(x['answer'])
    }) # type: ignore
    return data # type: ignore

dataset = get_gsm8k_questions()

# Reward functions
def correctness_reward_func(prompts, completions, answer, **kwargs) -> list[float]:
    responses = [completion[0]['content'] for completion in completions]
    q = prompts[0][-1]['content']
    extracted_responses = [extract_xml_answer(r) for r in responses]
    print('-'*20, f"Question:\n{q}", f"\nAnswer:\n{answer[0]}", f"\nResponse:\n{responses[0]}", f"\nExtracted:\n{extracted_responses[0]}")
    return [2.0 if r == a else 0.0 for r, a in zip(extracted_responses, answer)]

def int_reward_func(completions, **kwargs) -> list[float]:
    responses = [completion[0]['content'] for completion in completions]
    extracted_responses = [extract_xml_answer(r) for r in responses]
    return [0.5 if r.isdigit() else 0.0 for r in extracted_responses]

def strict_format_reward_func(completions, **kwargs) -> list[float]:
    """Reward function that checks if the completion has a specific format."""
    pattern = r"^<reasoning>\n.*?\n</reasoning>\n<answer>\n.*?\n</answer>\n$"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    return [0.5 if match else 0.0 for match in matches]

def soft_format_reward_func(completions, **kwargs) -> list[float]:
    """Reward function that checks if the completion has a specific format."""
    pattern = r"<reasoning>.*?</reasoning>\s*<answer>.*?</answer>"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    return [0.5 if match else 0.0 for match in matches]

def count_xml(text) -> float:
    count = 0.0
    if text.count("<reasoning>\n") == 1:
        count += 0.125
    if text.count("\n</reasoning>\n") == 1:
        count += 0.125
    if text.count("\n<answer>\n") == 1:
        count += 0.125
        count -= len(text.split("\n</answer>\n")[-1])*0.001
    if text.count("\n</answer>") == 1:
        count += 0.125
        count -= (len(text.split("\n</answer>")[-1]) - 1)*0.001
    return count

def xmlcount_reward_func(completions, **kwargs) -> list[float]:
    contents = [completion[0]["content"] for completion in completions]
    return [count_xml(c) for c in contents]

test-00000-of-00001.parquet:   0%|          | 0.00/613k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/3.37M [00:00<?, ?B/s]

Generating test split:   0%|          | 0/1319 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/7473 [00:00<?, ? examples/s]

Map:   0%|          | 0/7473 [00:00<?, ? examples/s]

<a name="Train"></a>
### Train the model

Now set up GRPO Trainer and all configurations!

In [13]:
from trl import GRPOConfig, GRPOTrainer
training_args = GRPOConfig(
    use_vllm = True, # use vLLM for fast inference!
    learning_rate = 5e-6,
    adam_beta1 = 0.9,
    adam_beta2 = 0.99,
    weight_decay = 0.1,
    warmup_ratio = 0.1,
    lr_scheduler_type = "cosine",
    optim = "adamw_8bit",
    logging_steps = 1,
    bf16 = is_bfloat16_supported(),
    fp16 = not is_bfloat16_supported(),
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 1, # Increase to 4 for smoother training
    num_generations = 8, # Decrease if out of memory
    max_prompt_length = 256,
    max_completion_length = 200,
    # num_train_epochs = 1, # Set to 1 for a full training run
    max_steps = 250,
    save_steps = 250,
    max_grad_norm = 0.1,
    report_to = "none", # Can use Weights & Biases
    output_dir = "outputs",
)

torch.distributed process group is initialized, but parallel_mode != ParallelMode.DISTRIBUTED. In order to use Torch DDP, launch your script with `python -m torch.distributed.launch


And let's run the trainer! If you scroll up, you'll see a table of rewards. The goal is to see the `reward` column increase!

You might have to wait 150 to 200 steps for any action. You'll probably get 0 reward for the first 100 steps. Please be patient!

| Step | Training Loss | reward    | reward_std | completion_length | kl       |
|------|---------------|-----------|------------|-------------------|----------|
| 1    | 0.000000      | 0.125000  | 0.000000   | 200.000000        | 0.000000 |
| 2    | 0.000000      | 0.072375  | 0.248112   | 200.000000        | 0.000000 |
| 3    | 0.000000      | -0.079000 | 0.163776   | 182.500000        | 0.000005 |


In [ ]:
trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        xmlcount_reward_func,
        soft_format_reward_func,
        strict_format_reward_func,
        int_reward_func,
        correctness_reward_func,
    ],
    args = training_args,
    train_dataset = dataset,
)
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 7,473 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 1 | Gradient Accumulation steps = 1
\        /    Total batch size = 1 | Total steps = 250
 "-____-"     Number of trainable parameters = 119,734,272


-------------------- Question:
艾哈迈德和艾米丽正在进行一场比赛，看谁能获得班上最好的成绩。共有 9 项作业，艾哈迈德在课堂上得了 91 分。 Emily 的得分为 92。最终作业的价值与所有其他作业的价值相同。艾米丽的期末作业得了 90 分。如果所有成绩均为整数，艾哈迈德击败艾米丽所需的最低成绩是多少？ 
Answer:
100 
Response:
<reasoning>
为了分析艾哈迈德和艾米丽的成绩，我们需要计算出完成所有作业后总分相同所对应的最低成绩。要进行比较，需要首先确定完成已知作业后总分是多少，然后找出需要满足的前提条件，即艾哈迈德的总分为艾米丽的总分加上或减去一个整数。

艾哈迈德在9项作业中得了91分，艾米丽有8项作业得了92分，另外一项得90分。那么艾米丽全部9项作业的总分为: \(92+92+92+92+92+92+92+92+90=630\)。

现在，我们需要计算艾哈迈德至少需要多少分才能超过艾米丽。因为已经知道艾米丽的得分63格，我们需要找到艾哈迈德至少需要比63大的最小整数。 
Extracted:
<reasoning>
为了分析艾哈迈德和艾米丽的成绩，我们需要计算出完成所有作业后总分相同所对应的最低成绩。要进行比较，需要首先确定完成已知作业后总分是多少，然后找出需要满足的前提条件，即艾哈迈德的总分为艾米丽的总分加上或减去一个整数。

艾哈迈德在9项作业中得了91分，艾米丽有8项作业得了92分，另外一项得90分。那么艾米丽全部9项作业的总分为: \(92+92+92+92+92+92+92+92+90=630\)。

现在，我们需要计算艾哈迈德至少需要多少分才能超过艾米丽。因为已经知道艾米丽的得分63格，我们需要找到艾哈迈德至少需要比63大的最小整数。


Step,Training Loss,reward,reward_std,completion_length,kl
1,0.000000,0.125000,0.000000,200.000000,0.000000
2,0.000000,0.134875,0.051568,189.875000,0.000000
3,0.000000,0.176625,0.094514,179.625000,0.000014
4,0.000000,0.146000,0.059397,191.750000,0.000013
5,0.000000,0.194375,0.209874,200.000000,0.000011
6,0.000000,0.105750,0.016087,200.000000,0.000013
7,0.000000,0.223375,0.087894,167.625000,0.000015
8,0.000000,0.381250,0.941186,195.250000,0.000011
9,0.000000,0.151750,0.105632,178.125000,0.000024
10,0.000000,0.195500,0.102343,166.000000,0.000019


-------------------- Question:
水箱上的水表显示水箱已充满 1/3 的水。为了填充水箱，添加了 16 加仑的水。水箱满后可容纳多少加仑的水？ 
Answer:
24 
Response:
<reasoning>
假设水箱的容量为 \(x\) 加仑。根据题意，水箱现满 1/3 充满，说明现在的水量为 \(x/3\) 加仑。通过添加 爸爸加 16 加仑的水后，使得水箱的水量达到了满状态。那么就可以列出等式：
\[x/3 + 16 = x\]

将等式转换为：
\[x/3 = 16\]

通过等式两边同时乘以 3 来解出 \(x\)：
\[x = 16 \times 3 = 48\]

因此，水箱的容量为 48 加仑。

根据这个计算，我们知道水箱满后能容纳的水的总量是 48 加仑。
...
<answer>
48 加仑
</answer> 
Extracted:
48 加仑
-------------------- Question:
学校食堂有15张桌子。每张桌子可容纳10人。通常，只有 1/10 的座位空着。通常有多少个座位？ 
Answer:
135 
Response:
<reasoning>
通常情况下，只有 1/10 的座位是空的，这意味着 9/10 的座位是被占用的。由于每张桌子可以容纳10个人，总共可以容纳的人数（即座位的总数）可以通过以下方式计算：首先计算所有桌子的总容量，即 15*10 人，然后乘以 (9/10) 以表示空座位占总座位的比例，即通常有多少座位。 
</reasoning>
<answer>
学校食堂通常有 135 个座位。即计算如下：15 张桌子，每张桌子可以容纳 10 个座位，总座位为 15 * 10 = 150 个座位。当只有 1/10 的座位空着时，也就是有 9/10 的座位被占用，即 150 * 9/10 = 1 
Extracted:
学校食堂通常有 135 个座位。即计算如下：15 张桌子，每张桌子可以容纳 10 个座位，总座位为 15 * 10 = 150 个座位。当只有 1/10 的座位空着时，也就是有 9/10 的座位被占用，即 150 * 9/10 = 1
-------------------- Question:
Fiona 一小时内完成了 36 道数学题。雪莉在同一时间内完成了两

Step,Training Loss,reward,reward_std,completion_length,kl
1,0.000000,0.125000,0.000000,200.000000,0.000000
2,0.000000,0.134875,0.051568,189.875000,0.000000
3,0.000000,0.176625,0.094514,179.625000,0.000014
4,0.000000,0.146000,0.059397,191.750000,0.000013
5,0.000000,0.194375,0.209874,200.000000,0.000011
6,0.000000,0.105750,0.016087,200.000000,0.000013
7,0.000000,0.223375,0.087894,167.625000,0.000015
8,0.000000,0.381250,0.941186,195.250000,0.000011
9,0.000000,0.151750,0.105632,178.125000,0.000024
10,0.000000,0.195500,0.102343,166.000000,0.000019


-------------------- Question:
拍卖会上展出的三辆旧车的生产日期不同。第一辆车生产于1970年，比第二辆车早了10年。第三辆车是在第二辆车制造出来20年后制造的。计算第三辆汽车的制造年份。 
Answer:
2000 
Response:
<reasoning>
根据题意，我们可以按顺序计算每辆汽车的生产年份。第一辆车生产于1970年，第二辆车比第一辆车早10年，所以第二辆车的生产年份是1970 - 10 = 1960年。第三辆车生产于第二辆车之后20年，所以第三辆车的生产年份是1960 + 20 = 1980年。
</reasoning>
<answer>
1980年
</answer> 
Extracted:
1980年
-------------------- Question:
周末在马里奥理发店理发的费用要贵 50%。如果马里奥周一为最后一次理发支付了 18 美元，那么前一天他会支付多少钱？ 
Answer:
27 
Response:
<reasoning>
根据题意，标准理发费用的 50% 等于差价，因此标准理发费用为 18 美元的一半。设标准理发费用为 \(x\) 美元，那么马里奥付的费用为 \(x + 0.5x = 1.5x\) 美元。我们可以通过等式 \(1.5x = 18\) 算出标准重量的理发费用，即 \(x = 18 / 1.5\)。求解得知，标准理发费用为 12 美元。因此，前一天理发的费用会是标准价格的一半，即 6 美元。
</reasoning>
<answer>
6 美元
</answer> 
Extracted:
6 美元
-------------------- Question:
珍妮特有一家销售狗和猫定制项圈的公司。如果制作一个狗项圈需要 18 英寸尼龙，制作一个猫项圈需要 10 英寸尼龙，那么制作 9 个狗项圈和 3 个猫项圈需要多少尼龙？ 
Answer:
192 
Response:
<reasoning>
珍妮特要制作的狗项圈总长度为 \(9 \times 18 = 162\) 英寸，猫项圈总长度为 \(3 \times 10 = 30\) 英寸。因此，需要的尼龙总长度为狗项圈和猫项圈长度之和，即 \(162 + 30 = 192\) 英寸。
</reasoning>
<answer>


<a name="Inference"></a>
### Inference
Now let's try the model we just trained! First, let's first try the model without any GRPO trained:

In [ ]:
text = tokenizer.apply_chat_template([
    {"role" : "user", "content" : "How many r's are in strawberry?"},
], tokenize = False, add_generation_prompt = True)

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.8,
    top_p = 0.95,
    max_tokens = 1024,
)
output = model.fast_generate(
    [text],
    sampling_params = sampling_params,
    lora_request = None,
)[0].outputs[0].text

output

And now with the LoRA we just trained with GRPO - we first save the LoRA first!

In [ ]:
model.save_lora("grpo_saved_lora")

Now we load the LoRA and test:

In [ ]:
text = tokenizer.apply_chat_template([
    {"role" : "system", "content" : SYSTEM_PROMPT},
    {"role" : "user", "content" : "How many r's are in strawberry?"},
], tokenize = False, add_generation_prompt = True)

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.8,
    top_p = 0.95,
    max_tokens = 1024,
)
output = model.fast_generate(
    text,
    sampling_params = sampling_params,
    lora_request = model.load_lora("grpo_saved_lora"),
)[0].outputs[0].text

output

Our reasoning model is much better - it's not always correct, since we only trained it for an hour or so - it'll be better if we extend the sequence length and train for longer!

<a name="Save"></a>
### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

[**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/drive/1WZDi7APtQ9VsvOrQSSC5DDtxq159j8iZ?usp=sharing)

In [ ]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "",
    )

Now, use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in llama.cpp or a UI based system like Jan or Open WebUI. You can install Jan [here](https://github.com/janhq/jan) and Open WebUI [here](https://github.com/open-webui/open-webui)

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/unsloth) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Llama 3.2 Conversational notebook. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(1B_and_3B)-Conversational.ipynb)
2. Saving finetunes to Ollama. [Free notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision finetuning - Radiography use case. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. See notebooks for DPO, ORPO, Continued pretraining, conversational finetuning and more on our [documentation](https://docs.unsloth.ai/get-started/unsloth-notebooks)!

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  Join Discord if you need help + ⭐️ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐️
</div>
